In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
import PIL
import tensorflow as tf
import requests

from tensorflow import keras
from tensorflow.keras import layers, models, applications

# Note : we are using TensorFlow Core v2.5.0, in TensorFlow Core v2.6.0 all the data 
# augmentation layers are part of tf.keras.layers
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adamax


TRAIN_DIR = "../raw_data/wikiart/wikiart-target_style-class_14-keepgenre_True-merge_style_m1-flat_False/train"
VAL_DIR = "../raw_data/wikiart/wikiart-target_style-class_14-keepgenre_True-merge_style_m1-flat_False/val"
TEST_DIR = "../raw_data/wikiart/wikiart-target_style-class_14-keepgenre_True-merge_style_m1-flat_False/test"

BATCH_SIZE = 128 # Hyper param, you can tune it
EPOCHS = 1000 # Large number, early stopping to stop training before this number
IMG_HEIGHT = 224 # VGG's dim
IMG_WIDTH = 224 # VGG's dim
NUM_CLASSES = 14 # Number of art styles

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=TRAIN_DIR,
    labels='inferred',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=True)

assert len(train_ds.class_names) == NUM_CLASSES

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=VAL_DIR,
    labels='inferred',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=True)

assert len(val_ds.class_names) == NUM_CLASSES

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=TEST_DIR,
    labels='inferred',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=True)

assert len(test_ds.class_names) == NUM_CLASSES

total_images_count = (int(len(list(train_ds)))+int(len(list(val_ds)))+int(len(list(test_ds))))*BATCH_SIZE


AUTOTUNE = tf.data.experimental.AUTOTUNE

# Optimizing the dataset by caching and prefetching the data
train_ds = train_ds.cache().shuffle(int(total_images_count)).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 46971 files belonging to 14 classes.
Found 5871 files belonging to 14 classes.
Found 5872 files belonging to 14 classes.


In [3]:
base_model = applications.VGG16(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=False, weights='imagenet')
base_model.trainable = False

In [4]:
# Add data augmentation layers to your model
data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1),
])

In [5]:
# Define the art style recognition model
inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs, outputs)

In [6]:
model.compile(optimizer=Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=5),
    ModelCheckpoint("art_style_recognition_model.h5", save_best_only=True),
    TensorBoard(log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")),
]

In [8]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/1000


2023-10-24 12:37:52.379671: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 16 of 58752
2023-10-24 12:38:02.480890: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 38 of 58752
2023-10-24 12:38:12.343766: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 59 of 58752
2023-10-24 12:38:22.784012: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 79 of 58752
2023-10-24 12:38:32.442338: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 99 of 58752
2023-10-24 12:38:42.351415: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 118 of 58752
2023-10-24 12:38:52.200619: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle 

367/367 [==============================] - 5189s 14s/step - loss: 1.7298 - accuracy: 0.4579 - val_loss: 1.5419 - val_accuracy: 0.5004 - lr: 0.0010
Epoch 2/1000


/Users/jiangtianhan/Desktop/You will get good one day/TheUltimateAi/notebook/.venv/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


367/367 [==============================] - 5078s 14s/step - loss: 1.3447 - accuracy: 0.5294 - val_loss: 1.3932 - val_accuracy: 0.5190 - lr: 0.0010
Epoch 3/1000
367/367 [==============================] - 5023s 14s/step - loss: 1.2529 - accuracy: 0.5595 - val_loss: 1.3421 - val_accuracy: 0.5445 - lr: 0.0010
Epoch 4/1000
367/367 [==============================] - 5109s 14s/step - loss: 1.1903 - accuracy: 0.5759 - val_loss: 1.3801 - val_accuracy: 0.5227 - lr: 0.0010
Epoch 5/1000
367/367 [==============================] - 5136s 14s/step - loss: 1.1384 - accuracy: 0.5958 - val_loss: 1.3380 - val_accuracy: 0.5403 - lr: 0.0010
Epoch 6/1000
367/367 [==============================] - 5100s 14s/step - loss: 1.0896 - accuracy: 0.6108 - val_loss: 1.3115 - val_accuracy: 0.5468 - lr: 0.0010
Epoch 7/1000
367/367 [==============================] - 5028s 14s/step - loss: 1.0556 - accuracy: 0.6206 - val_loss: 1.2853 - val_accuracy: 0.5502 - lr: 0.0010
Epoch 8/1000
367/367 [==============================]

In [9]:
model.save("art_style_recognition_final_model.h5")

In [11]:
test_loss, test_accuracy = model.evaluate(test_ds)
print("Test accuracy:", test_accuracy)

46/46 [==============================] - 567s 12s/step - loss: 1.3052 - accuracy: 0.5894
Test accuracy: 0.5894073843955994


In [1]:
test_predictions = model.predict(test_ds)

top5_classes = tf.math.top_k(test_predictions, k=5).indices
top5_probilities = tf.math.top_k(test_predictions, k=5).values

print("Top 5 classes for the first image:", top5_classes[0].numpy())
print("Probabilities for the top 5 classes for the first image:", top5_probilities[0].numpy())

NameError: name 'model' is not defined